In [ ]:
! pip install -U pip setuptools wheel

In [ ]:
! pip install -U spacy

In [ ]:
! python -m spacy download en_core_web_sm

### Using Set storage

In [1]:
location_cuisine_map = {
    "indian",
    "west indian",
    "thai",
    "sushi",
    "chinese",
    "carribean",
    "italian",
    "pub",
    "bbq",
    "portugese",
    "spanish",
    "french",
    "east european"
}

In [2]:
import spacy    
nlp = spacy.load("en_core_web_sm")
# given we aim to just match cuisines
# hence we need to only match details related to food - 
# adding custom words to stop words to remove non-informational content
# Since no-one would ever ask to go to a bad food place - it doesn't proide any informational detail
# in queries about food - generally the following terms would be very frequent and thus carry less information 
# and have a significantly lower tf-idf value

nlp.Defaults.stop_words |= {"nice","good","like", "love", "i", "we", "me", "where", "how", "which", "when", "you", "u", "tell", "can"}
required_pattern = {"food", "restaurants", "food-places", "hotels", "takeaway", "eatery", "pub", "eat", "drink"}

In [3]:
def generate_n_grams(n, word_list):
    if not isinstance(word_list, list):
        raise ValueError("Word list must be a list of words over which ngrams have to generated")
    ngrams = zip(*[word_list[i:] for i in range(n)]) # O(n)
    return [" ".join(ngram) for ngram in ngrams]

In [4]:
def extract_relevant_terms(input_string):
    # this is O(n)
    data = [token.text.lower() for token in nlp(input_string) if token.text.lower() not in nlp.Defaults.stop_words]
    
    # O(n)
    search_result = any(i for i in data if i in required_pattern)
    if not search_result: return None
    
    queries = generate_n_grams(n=2, word_list=data)
    return queries

In [5]:

def get_match(input_string):
    
    queries = extract_relevant_terms(input_string)
    if not queries:
        return None
    
    # worst case(o(n))
    for query in queries:
        if query in location_cuisine_map:
            return input_string, query
    return None

In [6]:
%%timeit
get_match("Which restaurants do West Indian food")

3.17 ms ± 308 µs per loop (mean ± std. dev. of 7 runs, 100 loops each)


In [7]:
get_match("Which restaurants do West Indian food")

('Which restaurants do West Indian food', 'west indian')

In [8]:
%%timeit
get_match("What is the weather today")

2.86 ms ± 47.1 µs per loop (mean ± std. dev. of 7 runs, 100 loops each)


In [9]:
get_match("What is the weather today")

In [10]:
import sys

In [11]:
sys.getsizeof(location_cuisine_map)

744

# Bloom Filter

1. once uttered, you generally want to remember the utterances. Elements once added in the bloomfilter cannot be removed, unless we use an invertible bloom filter (this can be implemented with more ease using a list)
2.  A bloom filter can have fale positives but no false negatives. In this situation, we can afford false positives because in a query of food, we can generally be given all related options
3. Once added, elements cannot be removed from the bloom filter - which again serves our purpose. You wouldn't remove data added to cuisine map generally


#### Trying to optimize space a little

In [12]:
import sys

In [13]:
set_data = set()
dict_data = dict()
list_data = list()
tuple_data = tuple()

In [14]:
sys.getsizeof(list_data)

72

In [15]:
sys.getsizeof(set_data)

232

In [16]:
sys.getsizeof(dict_data)

248

In [17]:
sys.getsizeof(tuple_data)

56

### Memory Details for a bloom filter and choice of data structure

- traditionally bloomfilter is a bitarray. We can use a list here to construct a bloom filter. The size of the filter (no. of sparse spaces in the filter and the number of elements in the filter) is directly proportional to the risk of collision in the filter
- I assume here that space is not a limitation. As shown below, a list should take about 8MB for 1 million strings. Given list uses heap memory - using it for implementing a bloom filter should be okay.

- Both list and tuple give same memory in terms of storing large number of string. We use list for the ease of adding elements to the bloom filter.

In [18]:

list_data = ['arandondomverylongsensentencefor memory check'] *1000000
tuple_data = tuple(list_data)

In [19]:
sys.getsizeof(tuple_data)*0.000001 # 8MB - original value of getszeof is in bytes

8.000055999999999

In [20]:
sys.getsizeof(list_data)*0.000001 

8.000072

### Other details about BloomFilter


1. False Positive Probablity - Given the size of filter m, and number of elements n- the false positive probablity is given by - 

$$P = (1 - [1 - \frac 1 m]^{kn})^k$$

2. Optimal Size of the filter: Given desired probablity of false positives and n being the number of elements that are to be placed in the filter, we have:

$$m = -\frac{n\ln p}{(\ln 2)^2}$$

3. No. of hash functions: Given a filter of size m and we have n elements to be inserted, the optimal number for hash functions is:  

$$k = \frac{m} {n}\ln2$$

4. We need fast independent hash functions which are uniformly distributed. Here I am using mm3. We can also create multiple hash functions using md5, sha1 and sha224 - however these can be computationally expensive

In [19]:
! pip install mmh3

In [21]:
from dsa.bloom_filter import BloomFilter

In [22]:
n = len(location_cuisine_map)
p = 0.01

In [23]:
blf = BloomFilter(n=n, p=p)

In [24]:
location_cuisine_map = {
    "indian",
    "west indian",
    "thai",
    "sushi",
    "chinese",
    "carribean",
    "italian",
    "pub",
    "bbq",
    "portugese",
    "spanish",
    "french",
    "east european"
}


for item in location_cuisine_map:
    blf.insert(item)

In [25]:
sys.getsizeof(blf.filter)* 0.000001 
# size around 150 MB for one million 
# words with 0.01 probability for false negatives

0.001064

In [26]:
def check_filter(input_string):
    queries = extract_relevant_terms(input_string)
    if not queries:
        return None
    
    # worst case(o(n))
    for query in queries:
        if blf.is_present(query):
            return input_string, query
    return None

In [27]:
%%timeit
result = check_filter("Which restaurants do West Indian food")

3.28 ms ± 256 µs per loop (mean ± std. dev. of 7 runs, 100 loops each)


In [31]:
check_filter("Which restaurants do West Indian food")

('Which restaurants do West Indian food', 'west indian')

In [29]:
%%timeit
check_filter("What is the weather today")

3.28 ms ± 250 µs per loop (mean ± std. dev. of 7 runs, 100 loops each)


In [30]:
check_filter("What is the weather today")

I think Bloom Filter gives benefits in memory with speed comparable to hash map look up

In [31]:
BloomFilter.num_hashes(size=10, n=5)

1

In [32]:
BloomFilter.get_length(n=10, p=0.25)

28

In [42]:
blf = BloomFilter(n=3, p=0.1)
blf.insert("Gondor")

In [46]:
blf.is_present("Isenguard")

False